In [1]:
%matplotlib inline

import geopandas as gpd

import pandas as pd
import os
import requests
import folium
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

In [2]:
import matplotlib.font_manager as fm

for font in fm.fontManager.ttflist:
    if font.name in ['AppleGothic', 'Malgun Gothic']:
        plt.rcParams['font.family'] = font.name
        break

plt.rcParams['font.family']

['Malgun Gothic']

In [19]:
#2018년 7월 승차일 기준 1~4일의 버스 카드태깅 정보를 담고있습니다.
#미터 / 분
TripChain = pd.read_csv('../Data/PJT001_TripChain.csv')

#17~18년 기준, 경기도 버스 정류장에 대한 정보를 담고있습니다.
# 성보님과 은솔님이 채워주신 데이터 바로 이용
StationTable = pd.read_csv('../Data/station_null.csv')

#2018년 7월 1~4일의 행정동별 이동 인구수 정보를 담고 있습니다.
AreaPeople = pd.read_csv('../Data/PJT001_sk_emd_od.csv')

#버스노선-정류장 매핑 테이블 정보 입니다.
RouteStationInfo = pd.read_csv('../Data/PJT001_routestationinfo.csv')

#
RouteStationMapping = pd.read_csv('../Data/PJT001_routestationmapping.csv')

In [50]:
TripChain.columns=['암호화카드번호', '트랜잭션ID', '환승횟수', '교통카드발행사ID',
       '총이용객수', '사용자구분', '교통수단CD1', '교통수단CD2',
       '교통수단CD3', '교통수단CD4', '교통수단CD5', '버스노선ID1',
       '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
       '차량ID1', '차량ID2', '차량ID3', '차량ID4',
       '차량ID5', '총통행거리', '총탑승시간', '총소요시간',
       '승차일시1', '승차일시2', '승차일시3', '승차일시4',
       '승차일시5', '하차일시1', '하차일시2', '하차일시3',
       '하차일시4', '하차일시5', '최초승차일시', '최종하차일시',
       '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
       '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3',
       '하차역ID4', '하차역ID5', '최초승차역ID', '최종하차역ID',
       '총이용금액', '수집건수', '트립체인완료코드']

In [6]:
TripChain.sample()

,암호화카드번호,트랜잭션ID,환승횟수,교통카드발행사ID,총이용객수,사용자구분,교통수단CD1,교통수단CD2,교통수단CD3,교통수단CD4,...,하차역ID1,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차역ID,최종하차역ID,총이용금액,수집건수,트립체인완료코드
797771,900110315330,47,2,9000901,1,1,500.0,500.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4160374.0,1250,2,;


In [20]:
StationTable.shape

(39457, 14)

In [10]:
RouteStationInfo.sample()

,seq,pr_station_id,bus_line_no,bus_line_no_seq,station_nm,station_id,mobile_no
25632,163983,241317006,19-2,64,레이크반도9차,233003133,55529.0


In [11]:
RouteStationMapping.sample()

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
1994,경기시내,오산교통,4107400,223000008,41067005,7


In [37]:
RouteStationMapping[RouteStationMapping['이비노선ID']==228000018]

# 이비노선ID == RouteStationInfo 의 pr_station_id
# 표준노선ID == TripChain의 버스노선ID 1~5

,구분,운수사명,운수사ID,이비노선ID,표준노선ID,노선명
709,경기시내,경남여객,4100600,228000018,41006260,10-4


In [58]:
TripChain[TripChain['버스노선ID1']==41006260.0 ]

,암호화카드번호,트랜잭션ID,환승횟수,교통카드발행사ID,총이용객수,사용자구분,교통수단CD1,교통수단CD2,교통수단CD3,교통수단CD4,...,하차역ID1,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차역ID,최종하차역ID,총이용금액,수집건수,트립체인완료코드
351351,900458235861,59,3,9000929,1,1,500.0,500.0,500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4116703.0,1550,3,;
423008,900399600864,23,3,9000929,1,1,500.0,500.0,500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4117067.0,2150,3,;
474719,900520797175,56,4,9000921,1,1,500.0,500.0,202.0,500.0,...,NaN,NaN,NaN,4108114.0,NaN,NaN,4108114.0,1850,4,;
503222,900026836857,26,4,3101001,1,1,500.0,500.0,500.0,500.0,...,NaN,NaN,NaN,4111686.0,NaN,NaN,4111686.0,2950,4,;
552852,900388380402,8,4,3101915,1,1,500.0,500.0,500.0,500.0,...,NaN,NaN,NaN,4111850.0,NaN,NaN,4111850.0,1650,4,;
599761,900481809000,81,4,9000922,1,1,500.0,500.0,202.0,500.0,...,NaN,NaN,NaN,4135034.0,NaN,NaN,4135034.0,1550,4,;
803957,100500886129,21,3,3104012,1,1,500.0,500.0,500.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,4150162.0,1550,3,;
1008017,900397793341,8,4,3101000,1,1,500.0,500.0,500.0,500.0,...,NaN,NaN,NaN,4117031.0,NaN,NaN,4117031.0,1950,4,;


In [ ]:
RouteStationMapping[RouteStationMapping['화성시']]

# 화성시 데이터만 뽑겠습니다

In [ ]:
정류소별

In [68]:
hwasung_station = StationTable[StationTable['시군명']=='화성시']
# 정류소 번호 중복 제거
StationTable.drop_duplicates(subset ="정류소번호", 
                     keep = 'first', inplace = True) 

In [69]:
hwasung_station.shape

(11243, 14)

In [70]:
hwasung_station.sample()

,Unnamed: 0,표준정류장ID,시군명,정류소명,정류소영문명,정류소번호,중앙차로여부,관할관청,위치,WGS84위도,WGS84경도,모바일정류장ID,이비카드정류장ID,station_id
22910,22910,233000980,화성시,모아.반도아파트,"Moa, Bando Apartment",36459.0,노변정류장,경기도 화성시,경기도 화성시 동탄면,37.202033,127.11445,[None None None None None None None None None ...,4130059.0,233000980


~~쓸모없는 짓~~

In [71]:
tmp = pd.merge(hwasung_station,RouteStationInfo, how='left', left_on = 'station_id', right_on = 'station_id')

In [72]:
tmp[(tmp['WGS84위도'].isnull())==True].shape

(1776, 20)

In [74]:
tmp.shape

(105131, 20)

In [75]:
tmp2 = tmp.dropna(subset=['WGS84위도'])

In [76]:
tmp2.shape

(103355, 20)

---

In [59]:
TripChain.iloc[0]

암호화카드번호         900079696430
트랜잭션ID                    56
환승횟수                       2
교통카드발행사ID            9000923
총이용객수                      1
사용자구분                      1
교통수단CD1                  500
교통수단CD2                  500
교통수단CD3                  NaN
교통수단CD4                  NaN
교통수단CD5                  NaN
버스노선ID1           4.1002e+07
버스노선ID2           4.1002e+07
버스노선ID3                  NaN
버스노선ID4                  NaN
버스노선ID5                  NaN
차량ID1            1.41772e+08
차량ID2            1.41772e+08
차량ID3                    NaN
차량ID4                    NaN
차량ID5                    NaN
총통행거리                  11170
총탑승시간                     25
총소요시간                     25
승차일시1            2.01807e+13
승차일시2            2.01807e+13
승차일시3                    NaN
승차일시4                    NaN
승차일시5                    NaN
하차일시1            2.01807e+13
하차일시2            2.01807e+13
하차일시3                    NaN
하차일시4                    NaN
하차일시5                    NaN
최초승차일시        

In [ ]:
시간대별로 특정 노선이 이용되는 ~
버스 라인마다의 특성
학교 가 들어가는 정류장을 지나가는 버스의 노선 확인 (활성화 여부도 함께 확인)
1. 주로 노인이 타는버스 - 저상버스
2. 주로 학생이 타는 버스
버스 몇인승인지 확인
정류소 이름 활용
환승설계 - 긴 노선을 짤라서 환승. 미니 메트로

# 운수사명 확인 후 화성 버스만 사용

In [12]:
RouteStationMapping['운수사명'].unique()

array(['경원여객M', '강화운수', '경기고속', '대원고속', '대원버스', '대원운수', '경남여객', '경원여객',
       '경진여객', '금강고속', '김포운수', '남양여객', '대명운수', '대양운수', '도원교통', '명성운수',
       '명진여객', '백성운수', '부천버스', '삼경운수', '삼영운수', '서울고속', '선진시내버스', '성남시내버스',
       '성우운수', '소신여객', '수원여객', '시흥교통', '신성여객', '신일여객', '연천교통', '용남고속',
       '진흥고속', '태화상운', '평안운수', '포천교통', '포천상운', '협진여객', '화영운수', '경기여객',
       '용남버스', '보영운수', '성광운수', '양주교통', '진명여객', '제부여객', '부일교통소사', '부일교통작동',
       '평택여객', '고양교통', '청우운수', '부일교통상동', '오산교통', '화성운수', '선진버스', '경기버스',
       '경기운수', '경기상운', '파주선진', '(주)가온누리엠', '(주)화성여객', '과천여객', '서현운수',
       '코레일네트웍스', '신성교통', '군포여객'], dtype=object)

In [13]:
TripChain['암호화카드번호'].nunique()

498764

# 필요한 데이터만 뽑은 csv 파일 생성 및 사용

```
환승 횟수가 많은 이용객 분석

필요한 것
TripChain > 이용객 정보
            암호화카드번호ID, 버스노선ID 1~5, 총통행거리, 총탑승시간,총소요시간, 승차역1~5, 하차역1~5, 
            최초승차일시, 최종하차일시, 최초승차역ID, 최종하차역ID, 총이용금액, 수집건수
RouteStationMapping > 이용객이 어떤 버스를 탔는지 확인
                      표준노선ID -> TripChain의 버스노선ID와 merge
                      이비노선ID, 노선명
RouteStationInfo > 이용객이 탄 버스 정류장이 어딘지 저장
```

In [111]:
TripChain_over3 = TripChain[TripChain['환승횟수']>=3]

In [112]:
merge1 = pd.merge(TripChain_over3[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID1', 
                        right_on = '표준노선ID')

In [113]:
merge1['최초승차역ID'] = merge1['승차역ID1']

In [114]:
merge1 = merge1.dropna(subset=['노선명'])
merge1.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,표준노선ID,이비노선ID,노선명
0,900079705567,41067109.0,NaN,41021002.0,NaN,NaN,24480,39,39,4197606.0,...,NaN,20180701075731,20180701085359,4197606.0,4116627.0,1550,3,41067109.0,223000032.0,701
2,900079981594,41061025.0,NaN,41425003.0,NaN,NaN,19971,71,71,4108111.0,...,NaN,20180701190828,20180701202740,4108111.0,9119276.0,1450,3,41061025.0,200000105.0,15
3,900030101425,41027009.0,41031009.0,NaN,NaN,NaN,12560,45,45,4100114.0,...,NaN,20180701124244,20180701140028,4100114.0,1713.0,1350,3,41027009.0,200000028.0,301
5,900030101881,41008148.0,NaN,28222001.0,NaN,NaN,64294,130,130,4131032.0,...,NaN,20180701185022,20180701210709,4131032.0,2806914.0,2350,3,41008148.0,233000083.0,30
6,900030101881,41008148.0,NaN,28222001.0,NaN,NaN,64294,130,130,4131032.0,...,NaN,20180701185022,20180701210709,4131032.0,2806914.0,2350,3,41008148.0,233000083.0,30


In [139]:
merge1.shape

(81415, 25)

In [115]:
del merge1['표준노선ID']

In [116]:
del merge1['이비노선ID']

In [117]:
merge1['버스노선ID1'] = merge1['노선명']

In [118]:
del merge1['노선명']

In [124]:
merge2 = pd.merge(merge1[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID2', 
                        right_on = '표준노선ID')

In [125]:
merge2.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,표준노선ID,이비노선ID,노선명
0,900079705567,701,NaN,41021002.0,NaN,NaN,24480,39,39,4197606.0,...,NaN,20180701075731,20180701085359,4197606.0,4116627.0,1550,3,NaN,NaN,NaN
1,900079981594,15,NaN,41425003.0,NaN,NaN,19971,71,71,4108111.0,...,NaN,20180701190828,20180701202740,4108111.0,9119276.0,1450,3,NaN,NaN,NaN
2,900030101425,301,41031009.0,NaN,NaN,NaN,12560,45,45,4100114.0,...,NaN,20180701124244,20180701140028,4100114.0,1713.0,1350,3,41031009.0,200000049.0,112
3,900030101881,30,NaN,28222001.0,NaN,NaN,64294,130,130,4131032.0,...,NaN,20180701185022,20180701210709,4131032.0,2806914.0,2350,3,NaN,NaN,NaN
4,900030101881,30,NaN,28222001.0,NaN,NaN,64294,130,130,4131032.0,...,NaN,20180701185022,20180701210709,4131032.0,2806914.0,2350,3,NaN,NaN,NaN


In [126]:
merge2['버스노선ID2'] = merge2['노선명']
merge2 = merge2.dropna(subset=['노선명'])
del merge2['노선명']
del merge2['표준노선ID']
del merge2['이비노선ID']
merge2.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수
2,900030101425,301,112,NaN,NaN,NaN,12560,45,45,4100114.0,...,NaN,NaN,NaN,NaN,20180701124244,20180701140028,4100114.0,1713.0,1350,3
5,900030102978,30-3,1009,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,NaN,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4
7,900480680100,13-1,M5107,11110052.0,11110055.0,NaN,45645,66,66,4108218.0,...,NaN,NaN,9034961.0,NaN,20180701144127,20180701155848,4108218.0,9034961.0,2800,4
11,900508818160,7,202,41031107.0,NaN,NaN,10930,24,24,4108196.0,...,NaN,NaN,NaN,NaN,20180701204621,20180701213623,4108196.0,4199077.0,1350,3
12,900351733805,13-5,400-4,41036128.0,NaN,NaN,16150,49,49,4108106.0,...,NaN,NaN,NaN,NaN,20180701145032,20180701161318,4108106.0,4102942.0,1450,3


In [127]:
merge2.shape

(51187, 25)

In [133]:
merge3 = pd.merge(merge2[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID3', 
                        right_on = '표준노선ID')

In [134]:
merge3.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,표준노선ID,이비노선ID,노선명
0,900030101425,301,112,NaN,NaN,NaN,12560,45,45,4100114.0,...,NaN,20180701124244,20180701140028,4100114.0,1713.0,1350,3,NaN,NaN,NaN
1,900030102978,30-3,1009,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,41031019.0,200000025.0,92
2,900030102978,30-3,1009,41031019.0,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,41031019.0,200000025.0,92
3,900480680100,13-1,M5107,11110052.0,11110055.0,NaN,45645,66,66,4108218.0,...,NaN,20180701144127,20180701155848,4108218.0,9034961.0,2800,4,NaN,NaN,NaN
4,900508818160,7,202,41031107.0,NaN,NaN,10930,24,24,4108196.0,...,NaN,20180701204621,20180701213623,4108196.0,4199077.0,1350,3,41031107.0,200000077.0,5-1


In [136]:
merge3 = merge3.dropna(subset=['노선명'])
merge3['버스노선ID3'] = merge3['노선명']
del merge3['노선명']
del merge3['표준노선ID']
del merge3['이비노선ID']

In [138]:
merge3.shape

(40476, 25)

### 환승횟수가 4인 데이터

In [141]:
TripChain_4 = merge3[merge3['수집건수']==4]

In [142]:
TripChain_4.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수
1,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,NaN,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4
2,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,NaN,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4
36,900480674905,51,720-1,700-2,41419002.0,NaN,14050,42,42,4110270.0,...,NaN,NaN,4115063.0,NaN,20180701074728,20180701091223,4110270.0,4115063.0,1350,4
41,900469148309,98,777,310,41006241.0,NaN,12030,41,41,4116947.0,...,NaN,NaN,NaN,NaN,20180701144949,20180701160447,4116947.0,NaN,1350,4
42,900469148309,98,777,310,41006241.0,NaN,12030,41,41,4116947.0,...,NaN,NaN,NaN,NaN,20180701144949,20180701160447,4116947.0,NaN,1350,4


In [143]:
merge4 = pd.merge(TripChain_4[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID4', 
                        right_on = '표준노선ID')
merge4.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,표준노선ID,이비노선ID,노선명
0,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,41036110.0,200000097.0,2-1
1,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,41036110.0,200000097.0,2-1
2,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,41036110.0,200000097.0,2-1
3,900030102978,30-3,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4,41036110.0,200000097.0,2-1
4,900480674905,51,720-1,700-2,41419002.0,NaN,14050,42,42,4110270.0,...,NaN,20180701074728,20180701091223,4110270.0,4115063.0,1350,4,NaN,NaN,NaN


In [144]:
merge4['버스노선ID1'] = merge4['노선명']
merge4 = merge4.dropna(subset=['노선명'])
del merge4['노선명']
del merge4['표준노선ID']
del merge4['이비노선ID']

In [145]:
merge4.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID2,하차역ID3,하차역ID4,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수
0,900030102978,2-1,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,NaN,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4
1,900030102978,2-1,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,NaN,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4
2,900030102978,2-1,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,NaN,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4
3,900030102978,2-1,1009,92,41036110.0,NaN,62600,165,165,4176873.0,...,NaN,NaN,4116855.0,NaN,20180701150518,20180701182312,4176873.0,4116855.0,3100,4
5,900469148309,66,777,310,41006241.0,NaN,12030,41,41,4116947.0,...,NaN,NaN,NaN,NaN,20180701144949,20180701160447,4116947.0,NaN,1350,4


In [146]:
merge4.shape

(6041, 25)

### 환승횟수가 5인 데이터

In [147]:
TripChain_5 = merge3[merge3['수집건수']==5]

In [148]:
merge5 = pd.merge(TripChain_5[['암호화카드번호', '버스노선ID1', '버스노선ID2', '버스노선ID3', '버스노선ID4', '버스노선ID5',
          '총통행거리', '총탑승시간','총소요시간', '승차역ID1', '승차역ID2', '승차역ID3', '승차역ID4',
           '승차역ID5', '하차역ID1', '하차역ID2', '하차역ID3', '하차역ID4', '하차역ID5',
           '최초승차일시', '최종하차일시', '최초승차역ID', '최종하차역ID', '총이용금액', '수집건수']],
RouteStationMapping[['표준노선ID', '이비노선ID', '노선명']], how = 'left', left_on = '버스노선ID5', 
                        right_on = '표준노선ID')
merge5.head()

,암호화카드번호,버스노선ID1,버스노선ID2,버스노선ID3,버스노선ID4,버스노선ID5,총통행거리,총탑승시간,총소요시간,승차역ID1,...,하차역ID5,최초승차일시,최종하차일시,최초승차역ID,최종하차역ID,총이용금액,수집건수,표준노선ID,이비노선ID,노선명
0,900097650068,9-3,7770,64,41031030.0,41008007.0,54250,88,88,4103271.0,...,4116659.0,20180701092638,20180701112742,4103271.0,4116659.0,2900,5,41008007.0,233000031.0,7770
1,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,41006241.0,228000001.0,66
2,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,41006241.0,228000001.0,66
3,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,41006241.0,228000001.0,66
4,900029059863,4-1,2-1,98,41031040.0,41006241.0,15080,58,58,4170101.0,...,4116887.0,20180701141312,20180701161148,4170101.0,4116887.0,1030,5,41006241.0,228000001.0,66


In [149]:
merge5['버스노선ID1'] = merge5['노선명']
merge5 = merge5.dropna(subset=['노선명'])
del merge5['노선명']
del merge5['표준노선ID']
del merge5['이비노선ID']
merge5.shape

(1723, 25)

### 환승횟수 3개 이상인 데이터들 다같이 concat

# 정류소명 이용